In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import FunctionLib
import math
#%matplotlib widget
basePath ='./sims.TNG/TNG50-1/output'
from tqdm import tqdm
import gc
import os

In [3]:
basePath = './sims.TNG/TNG50-1/output'
initial_snapshot = 84
end_snapshot = 99
h=0.6774
Critial_Density=1.27E-8  #E10M_Sum/kpc^3
Critial_Density_h=Critial_Density/h**2  #E10M_Sum/(kpc/h)^3

In [40]:
halos_fields = ['GroupFirstSub', 'GroupNsubs', 'GroupMass', 'GroupPos']
subhalos_fields = ['SubhaloGrNr', 'SubhaloMass', 'SubhaloPos']
tree_fields = ['SubfindID', 'SnapNum']

gas_fields=['Coordinates','Masses','Velocities']
dm_fields=['Coordinates','Velocities']
stars_fields=['Coordinates','Masses','Velocities']
bh_fields=['Coordinates','Masses','Velocities']
tree_fields=['SubfindID','SnapNum','Mass']

In [50]:
def Cal_M_200_From_R_200(R_200,Comoving_Critial_Density):
    
    M_200=200*Comoving_Critial_Density*4/3*3.14*R_200**3
    return M_200

def R_200_Calculation(snapshot_num,subhalos_data,subhalo_id,Comoving_Critial_Density):
    
    subhalo_comoving_position=subhalos_data['SubhaloPos'][subhalo_id]
    subhalo_comoving_velocity=subhalos_data['SubhaloVel'][subhalo_id]
    subhalo_spin=subhalos_data['SubhaloSpin'][subhalo_id]
    subhalo_mass=subhalos_data['SubhaloMass'][subhalo_id]
    
    gas_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,0,fields=gas_fields)
    dm_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,1,fields=dm_fields)
    stars_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,4,fields=stars_fields)
    bh_data_tensor=il.snapshot.loadSubhalo(basePath,snapshot_num,subhalo_id,5,fields=bh_fields)
    
    #if (len(gas_data_tensor)==1):
     #   return 0
    
    precision = 1
    max_iter = 100
    left, right = 0, 1000
    
    for i in range(max_iter):

        mid = (left + right) / 2
        
        mass_in=total_mass_in_radius(gas_data_tensor,dm_data_tensor,stars_data_tensor,bh_data_tensor,subhalo_comoving_position,mid)
        
        density=3*mass_in/(4*3.14*mid**3)
        
        if density > 200*Comoving_Critial_Density:
            left = mid

        else:
            right = mid
        

        if abs(right - left) < precision or i == max_iter - 1:
            R_200 = (left + right) / 2
            break
    
    M_200=Cal_M_200_From_R_200(R_200,Comoving_Critial_Density)
    
    return R_200,M_200

def baryon_mass_in_radius(Center,Radius,Coordinates,Masses):
    dist_to_com = np.sqrt(np.sum((Coordinates - Center)**2, axis=1))
    within_radius_idx = np.where(dist_to_com <= Radius)[0]
    total_mass_within_radius = np.sum(Masses[within_radius_idx])
    return total_mass_within_radius

def dm_mass_in_radius(Center,Radius,Coordinates):
    dist_to_com = np.sqrt(np.sum((Coordinates - Center)**2, axis=1))
    within_radius_idx = np.where(dist_to_com <= Radius)[0]
    total_mass_within_radius = m_dm*within_radius_idx.shape[0]*h
    return total_mass_within_radius

def total_mass_in_radius(gas_data_tensor,dm_data_tensor,stars_data_tensor,bh_data_tensor,Center,Radius):
    
    
    
    gas_mass_in=baryon_mass_in_radius(Center,Radius,gas_data_tensor['Coordinates'],gas_data_tensor['Masses'])
    dm_mass_in=dm_mass_in_radius(Center,Radius,dm_data_tensor['Coordinates'])
    stars_mass_in=baryon_mass_in_radius(Center,Radius,stars_data_tensor['Coordinates'],stars_data_tensor['Masses'])
    bh_mass_in=baryon_mass_in_radius(Center,Radius,bh_data_tensor['Coordinates'],bh_data_tensor['Masses'])
    
    total_mass_in=gas_mass_in+dm_mass_in+stars_mass_in+bh_mass_in
    
    return total_mass_in

def Binary_DataIter(Data_Set1,Data_Set2):
    for i in range(0,min(len(Data_Set1),len(Data_Set2))):
        yield Data_Set1[i],Data_Set2[i]
        
def cal_id_at_Z_from_99(Subhalo_Id_0,Z):#Z is the snapshot
    subfindid_snapnum_tree=il.sublink.loadTree(basePath, 99, Subhalo_Id_0, fields=tree_fields, onlyMPB=True)
    subfind_id=subfindid_snapnum_tree['SubfindID']
    snapshot_num=subfindid_snapnum_tree['SnapNum']
    index=np.where(snapshot_num==Z)[0]
    return subfind_id[index][0]

In [6]:
halos_data = il.groupcat.loadHalos(basePath, initial_snapshot, fields=halos_fields)
subhalos_data = il.groupcat.loadSubhalos(basePath, initial_snapshot, fields=subhalos_fields)

In [7]:
first_subhalo_id = halos_data['GroupFirstSub']
n_subhalos = halos_data['GroupNsubs']
group_mass = halos_data['GroupMass']
subhalos_grnr = subhalos_data['SubhaloGrNr']
subhalos_mass = subhalos_data['SubhaloMass']

In [8]:
galaxy_mass_inf = 30
galaxy_mass_sup = 500
condition = (group_mass >= galaxy_mass_inf) & (group_mass <= galaxy_mass_sup)
mw_like_index = np.where(condition)[0]

In [9]:
Host_Index_List=[]

for i in tqdm(mw_like_index):
    first_subhalo_index = first_subhalo_id[i]
    if first_subhalo_index == -1:
        continue
    if n_subhalos[i] == 1:
        continue

    mass_array = np.array([])
    for j in range(n_subhalos[i]):
        subhalo_id = first_subhalo_index + j
        mass_array = np.append(mass_array, subhalos_mass[subhalo_id])

    host_id = first_subhalo_index + np.argsort(mass_array)[-1]
    
    Host_Index_List.append(host_id)


100%|██████████| 482/482 [00:02<00:00, 217.13it/s]


In [11]:
spin_parameter_matrix=np.zeros(len(Host_Indice_Array),(abs(snapshot_end-snapshot_init + 1)))
for current_snapshot in range(init_snapshot, end_snapshot):
    
    header=il.groupcat.loadHeader(basePath, current_snapshot)
    scale_factor=header['Time']
    comoving_critial_density=Critial_Density_h*(scale_factor**2)
        
    subhalos_data=il.groupcat.loadSubhalos(basePath,current_snapshot,fields=subhalo_fields)
    
    for subhalo_index_0 in Host_Index_List:
        
        subhalo_id=cal_id_at_Z_from_99(subhalo_id_0,current_snapshot)

        R_200,M_200=R_200_Calculation(current_snapshot,subhalos_data,subhalo_id,comoving_critial_density)
        
        #Matrix append

In [53]:
end_snapshot=99
init_snapshot=70
subhalo_fields=['SubhaloPos','SubhaloVel','SubhaloSpin','SubhaloCM','SubhaloMass']
r200_matrix=np.zeros((len(Host_Index_List),(abs(end_snapshot-init_snapshot+1))))
for current_snapshot in range(end_snapshot,init_snapshot-1,-1):
    
    header=il.groupcat.loadHeader(basePath, current_snapshot)
    scale_factor=header['Time']
    comoving_critial_density=Critial_Density_h*(scale_factor**2)
        
    subhalos_data=il.groupcat.loadSubhalos(basePath,current_snapshot,fields=subhalo_fields)
    
    for host_id_0 in tqdm(Host_Indice_Array):
        host_index=np.where(Host_Indice_Array==host_id_0)[0][0]
        subhalo_id=cal_id_at_Z_from_99(host_id_0,current_snapshot)
        
        
        R_200,M_200=R_200_Calculation(current_snapshot,subhalos_data,subhalo_id,comoving_critial_density)
        
        r200_matrix[host_index][end_snapshot-current_snapshot]=R_200
        
    gc.collect()
        
np.save('r_200.npy',r200_matrix)

  0%|          | 0/482 [00:00<?, ?it/s]


TypeError: 'int' object is not iterable

In [36]:
gc.collect()

5247

In [46]:
subhalo_id

300734

In [47]:
gas_data_tensor=il.snapshot.loadSubhalo(basePath,99,300734,0,fields=gas_fields)

In [49]:
len(gas_data_tensor)

1